Web Scraping Advanced Football Statistics - Understat

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
from bs4 import BeautifulSoup
import json

In [3]:
# create urls for all seasons of all leagues
base_url = 'https://understat.com/league'
leagues = ['La_liga', 'EPL', 'Bundesliga', 'Serie_A', 'Ligue_1', 'RFPL']
seasons = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

In [4]:
# Starting with latest data for Spanish league, because I'm a Barcelona fan
url = base_url+'/'+leagues[0]+'/'+seasons[4]
res = requests.get(url)
soup = BeautifulSoup(res.content, "lxml")

# Based on the structure of the webpage, I found that data is in the JSON variable, under <script> tags
scripts = soup.find_all('script')

# Check our <script> tags
# for el in scripts:
#   print('*'*50)
#   print(el.text)

In [5]:
string_with_json_obj = ''

# Find data for teams
for el in scripts:
    if 'teamsData' in str(el):
        string_with_json_obj = str(el).strip()
      
# print(string_with_json_obj)

# strip unnecessary symbols and get only JSON data
ind_start = string_with_json_obj.index("('")+2
ind_end = string_with_json_obj.index("')")
json_data = string_with_json_obj[ind_start:ind_end]

json_data = json_data.encode('utf8').decode('unicode_escape')

In [6]:
# convert JSON data into Python dictionary
data = json.loads(json_data)
print(data.keys())
print('='*50)
print(data['138'].keys())
print('='*50)
print(data['138']['id'])
print('='*50)
print(data['138']['title'])
print('='*50)
print(data['138']['history'][0])

# Print pretty JSON data to check out what we have there
# s = json.dumps(data, indent=4, sort_keys=True)
# print(s)

dict_keys(['138', '140', '141', '142', '143', '145', '146', '147', '148', '150', '151', '152', '153', '154', '156', '158', '159', '223', '231', '232'])
dict_keys(['id', 'title', 'history'])
138
Sevilla
{'h_a': 'a', 'xG': 3.34997, 'xGA': 2.39239, 'npxG': 3.34997, 'npxGA': 1.64976, 'ppda': {'att': 252, 'def': 19}, 'ppda_allowed': {'att': 263, 'def': 20}, 'deep': 12, 'deep_allowed': 5, 'scored': 4, 'missed': 1, 'xpts': 1.9829999999999999, 'result': 'w', 'date': '2018-08-19 21:15:00', 'wins': 1, 'draws': 0, 'loses': 0, 'pts': 3, 'npxGD': 1.70021}


In [7]:
# Get teams and their relevant ids and put them into separate dictionary
teams = {}
for id in data.keys():
  teams[id] = data[id]['title']

In [8]:
# EDA to get a feeling of how the JSON is structured
# Column names are all the same, so we just use first element
columns = []
# Check the sample of values per each column
values = []
for id in data.keys():
  columns = list(data[id]['history'][0].keys())
  values = list(data[id]['history'][0].values())
  break

print(columns)
print(values)

['h_a', 'xG', 'xGA', 'npxG', 'npxGA', 'ppda', 'ppda_allowed', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'result', 'date', 'wins', 'draws', 'loses', 'pts', 'npxGD']
['a', 3.34997, 2.39239, 3.34997, 1.64976, {'att': 252, 'def': 19}, {'att': 263, 'def': 20}, 12, 5, 4, 1, 1.9829999999999999, 'w', '2018-08-19 21:15:00', 1, 0, 0, 3, 1.70021]


In [9]:
sevilla_data = []
for row in data['138']['history']:
  sevilla_data.append(list(row.values()))
  
df = pd.DataFrame(sevilla_data, columns=columns)
df.head(2)

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD
0,a,3.34997,2.392390,3.34997,1.649760,"{'att': 252, 'def': 19}","{'att': 263, 'def': 20}",12,5,4,1,1.9830,w,2018-08-19 21:15:00,1,0,0,3,1.700210
1,h,1.97161,0.671429,1.97161,0.671429,"{'att': 262, 'def': 16}","{'att': 237, 'def': 26}",11,3,0,0,2.3331,d,2018-08-26 21:15:00,0,1,0,1,1.300181


In [10]:
# Getting data for all teams
dataframes = {}
for id, team in teams.items():
  teams_data = []
  for row in data[id]['history']:
    teams_data.append(list(row.values()))
    
  df = pd.DataFrame(teams_data, columns=columns)
  dataframes[team] = df
  print('Added data for {}.'.format(team))

Added data for Sevilla.
Added data for Real Sociedad.
Added data for Espanyol.
Added data for Getafe.
Added data for Atletico Madrid.
Added data for Rayo Vallecano.
Added data for Valencia.
Added data for Athletic Club.
Added data for Barcelona.
Added data for Real Madrid.
Added data for Levante.
Added data for Celta Vigo.
Added data for Real Betis.
Added data for Villarreal.
Added data for Eibar.
Added data for Alaves.
Added data for Leganes.
Added data for Girona.
Added data for Real Valladolid.
Added data for SD Huesca.


In [11]:
# Sample check of our newly created DataFrame
dataframes['Barcelona'].head(2)

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD
0,h,3.26753,0.248353,3.26753,0.248353,"{'att': 118, 'def': 17}","{'att': 407, 'def': 13}",20,0,3,0,2.9009,w,2018-08-18 23:15:00,1,0,0,3,3.019177
1,a,1.20392,0.510742,1.20392,0.510742,"{'att': 163, 'def': 16}","{'att': 316, 'def': 15}",15,4,1,0,1.9865,w,2018-08-25 23:15:00,1,0,0,3,0.693178


#### Manipulations to make data as in the original source

In [12]:
for team, df in dataframes.items():
  dataframes[team]['ppda_coef'] = dataframes[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
  dataframes[team]['oppda_coef'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
  
# And check how our new dataframes look based on Sevilla dataframe
dataframes['Sevilla'].head(2)

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,...,xpts,result,date,wins,draws,loses,pts,npxGD,ppda_coef,oppda_coef
0,a,3.34997,2.392390,3.34997,1.649760,"{'att': 252, 'def': 19}","{'att': 263, 'def': 20}",12,5,4,...,1.9830,w,2018-08-19 21:15:00,1,0,0,3,1.700210,13.263158,13.150000
1,h,1.97161,0.671429,1.97161,0.671429,"{'att': 262, 'def': 16}","{'att': 237, 'def': 26}",11,3,0,...,2.3331,d,2018-08-26 21:15:00,0,1,0,1,1.300181,16.375000,9.115385


In [13]:
cols_to_sum = ['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'wins', 'draws', 'loses', 'pts', 'npxGD']
cols_to_mean = ['ppda_coef', 'oppda_coef']

In [14]:
frames = []
for team, df in dataframes.items():
  sum_data = pd.DataFrame(df[cols_to_sum].sum()).transpose()
  mean_data = pd.DataFrame(df[cols_to_mean].mean()).transpose()
  final_df = sum_data.join(mean_data)
  final_df['team'] = team
  final_df['matches'] = len(df)
  frames.append(final_df)
  
full_stat = pd.concat(frames)

In [15]:
full_stat = full_stat[['team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'npxG', 'xGA', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts']]
full_stat.sort_values('pts', ascending=False, inplace=True)
full_stat.reset_index(inplace=True, drop=True)
full_stat['position'] = range(1,len(full_stat)+1)

In [16]:
full_stat['xG_diff'] = full_stat['xG'] - full_stat['scored']
full_stat['xGA_diff'] = full_stat['xGA'] - full_stat['missed']
full_stat['xpts_diff'] = full_stat['xpts'] - full_stat['pts']